#### Date: Jun 2019 (*Review: March 2024*)

<br>Programmer: Christian Dittmar, Yiğitcan Özer

This is the demo script which illustrates the main functionalities of the 'NMF toolbox'. For a detailed description we refer to [1,2] (see References below).

#### The notebook proceeds in the following steps:
<br>1. It loads an example audio file containing drums and melodic instruments
<br>2. It computes the STFT of the audio data.
<br>3. It applies KAM and NMF as described in [2], with score-informed initialization of the components
<br>4. It visualizes the decomposition results.
<br>5. It resynthesizes the separated audio streams and saves them as wav files to the hard drive.

### Initialization

In [ ]:
import os
import numpy as np
import soundfile as sf
import IPython.display as ipd

from libnmfd.core.nmfconv import conv_model, \
    init_activations, init_templates, nmfd
from libnmfd.dsp.algorithms import hpss_kam_fitzgerald
from libnmfd.dsp.filters import alpha_wiener_filter
from libnmfd.dsp.transforms import forward_stft, inverse_stft, log_freq_log_mag
from libnmfd.utils import make_monaural, pcm_int16_to_float32np
from libnmfd.utils.core_utils import percussiveness_estimation, visualize_components_kam, visualize_components_nmf
from libnmfd.utils.core_utils import drum_specific_soft_constraints_nmf

INPUT_DIR = 'data/'
OUT_DIR = 'output/'

# create the output directory if it doesn't exist
if not os.path.isdir(OUT_DIR):
    os.makedirs(OUT_DIR)

filename = 'runningExample_IGotYouMixture.wav'

### 1. Load the audio signal

In [ ]:
# read signal

x, fs = sf.read(file=os.path.join(INPUT_DIR, filename),dtype=np.float32)

# make monaural if necessary
x = make_monaural(x)

# read corresponding transcription files
melody_transcription = np.loadtxt(os.path.join(INPUT_DIR, 'runningExample_IGotYouMelody.txt'))
drums_transcription = np.loadtxt(os.path.join(INPUT_DIR, 'runningExample_IGotYouDrums.txt'))

### 2. Compute STFT

In [ ]:
# spectral parameters
BLOCK_SIZE = 2048
HOP_SIZE = 512

# STFT computation
X, A, P = forward_stft(x, block_size=BLOCK_SIZE, hop_size=HOP_SIZE, reconst_mirror=True, append_frames=True)

# get dimensions and time and freq resolutions
num_bins, num_frames = X.shape
time_res = HOP_SIZE / fs
freq_res = fs / BLOCK_SIZE

# get logarithmically-spaced frequency axis version for visualization purposes
log_freq_log_mag_A, log_freq_axis = log_freq_log_mag(A=A, freq_res=freq_res)
num_log_bins = len(log_freq_axis)

### 3. Apply KAM-based Harmonic Percussive Separation

In [ ]:
# set common parameters
num_iter_kam = 30
kam_A, kern, kern_ord = hpss_kam_fitzgerald(X=A,
                                            num_iter=num_iter_kam, 
                                            kern_dim=13)

# visualize
fh1 = visualize_components_kam(kam_A, time_res=time_res, freq_res=freq_res, font_size=14)

# save result
fh1.savefig(os.path.join(OUT_DIR, 'demoDrumExtractionKAM_NMF_percThreshold_KAM.png'))

In [ ]:
audios = []

# resynthesize KAM results
for k in range(2):
    Y = kam_A[k] * np.exp(1j * P);
    y, _ = inverse_stft(X=Y, block_size=BLOCK_SIZE, hop_size=HOP_SIZE, reconst_mirror=True,
                        append_frames=True, num_samp=len(x))
    audios.append(y)
    # save result
    out_filepath = os.path.join(OUT_DIR,
                                'demoDrumExtractionKAM_NMF_percThreshold_KAM_component_{}_extracted_from_{}'.format(k, filename))

    sf.write(file=out_filepath, data=y, samplerate=fs)

#### Input audio mixture

In [ ]:
ipd.Audio(x, rate=fs)

#### Percussive component based on KAM

In [ ]:
ipd.Audio(audios[0].T, rate=fs)

#### Harmonic component based on KAM

In [ ]:
ipd.Audio(audios[1].T, rate=fs)

In [ ]:
# concatenate new NMF target
V = np.concatenate([kam_A[0], kam_A[1]])
num_double_bins = V.shape[0]

# prepare matrix to revert concatenation,
accu_mat = np.concatenate([np.eye(num_bins), np.eye(num_bins)], axis=1)

### 4. Apply score-informed NMF to KAM-based target

In [ ]:
# set common parameters
num_iter_nmf = 30 #  in the score-informed case, less iterations are necessary
num_template_frames = 1 #  this can also be adjusted upwards

# generate score-informed templates for the melodic part
pitched_W = init_templates(num_bins=num_bins,
                           num_template_frames=num_template_frames,
                           freq_res=freq_res,
                           pitches=melody_transcription[:, 1], 
                           strategy='pitched')

# generate score-informed activations for the melodic part
pitched_H = init_activations(num_frames=num_frames,
                             time_res=time_res,
                             pitches=melody_transcription[:, 1],
                             onsets=melody_transcription[:, 0],
                             durations=melody_transcription[:, 2],
                             strategy='pitched')

# generate score-informed activations for the drum part
drums_H = init_activations(num_frames=num_frames,
                           time_res=time_res,
                           onsets=drums_transcription[:, 0],
                           drums=drums_transcription[:, 1],
                           decay=0.75,
                           strategy='drums')


num_comp_drum = drums_H.shape[0]
drums_W = init_templates(num_bins=num_bins, strategy='drums', num_template_frames=num_template_frames)

In [ ]:
# join drum and pitched initialization
init_H = np.concatenate([drums_H, pitched_H], axis=0)

# now get the total number of components
num_comp = init_H.shape[0]

# fill remaining parts of templates with random values
# create joint templates
init_W = list()
informed_weight = 5

# fill missing template parts with noise
for k in range(num_comp):
    if k < num_comp_drum:
        init_W.append(np.concatenate([informed_weight * drums_W[k]/drums_W[k].max().reshape(-1, 1), 
                                      np.random.rand(num_bins, num_template_frames)], axis=0))
    else:
        init_W.append(np.concatenate([np.random.rand(num_bins, num_template_frames), 
                                     informed_weight * pitched_W[k-num_comp_drum]/pitched_W[k-num_comp_drum].max()],
                                     axis=0))

    # normalize to unit sum
    init_W[k] /= init_W[k].sum()

In [ ]:
# NMFD core method
nmfd_W, nmfd_H, _, _, tensor_W = nmfd(V=V, 
                                      num_comp=num_comp, 
                                      num_frames=num_frames, 
                                      num_iter=num_iter_nmf,
                                      num_template_frames=num_template_frames,
                                      init_W=init_W,
                                      init_H=init_H,
                                      num_bins=num_double_bins,
                                      # set soft constraint parameters
                                      func_preprocess=drum_specific_soft_constraints_nmf,
                                      kern=kern,
                                      decay=0.75)


# set percussiveness to ground truth information
perc_weight = np.concatenate([np.ones((1, len(drums_W))), 
                              np.zeros((1, len(pitched_W)))], 
                             axis=1)

# compute separate models for percussive and harmonic part
Vp = conv_model(W=tensor_W, H=np.diag(perc_weight[0]) @ nmfd_H)
Vh = conv_model(W=tensor_W, H=np.diag(1-perc_weight[0]) @ nmfd_H)
               
# accumulate back to original spectrum, reverting the stacking
# this step is described in the last paragraph of sec. 2.4 in [2]
Ap = accu_mat @ Vp
Ah = accu_mat @ Vh

# alpha-Wiener filtering
nmfd_A, _ = alpha_wiener_filter(mixture_X=A, source_A=[Ap, Ah], alpha=1.0)

In [ ]:
# visualize results
# create reduced version of templates for visualization
nmfdW_vis = list()
for nmfdW_curr in nmfd_W:
    nmfdW_curr = accu_mat @ nmfdW_curr
    nmfdW_vis.append(nmfdW_curr)

fh2, _ = visualize_components_nmf(V=A, 
                                  W=nmfdW_vis, 
                                  H=nmfd_H, 
                                  comp_V=nmfd_A, 
                                  freq_res=freq_res, 
                                  time_res=time_res, 
                                  font_size=14);

# save result
fh2.savefig(os.path.join(OUT_DIR, 'demoDrumExtractionKAM_NMF_scoreInformed_NMF.png'))

In [ ]:
audios = []

# resynthesize results of NMF with soft constraints and score information
for k in range(2):
    Y = nmfd_A[k] * np.exp(1j * P);
    y, _ = inverse_stft(X=Y,
                        block_size=BLOCK_SIZE,
                        hop_size=HOP_SIZE,
                        reconst_mirror=True,
                        append_frames=True,
                        num_samp=len(x))

    # save result
    out_filepath = os.path.join(OUT_DIR,
                                'demoDrumExtractionKAM_NMF_scoreInformed_NMF_component_{}_extracted_from_{}'.format(k, filename))

    sf.write(file=out_filepath, data=y, samplerate=fs)

    audios.append(y)

#### Input audio mixture

In [ ]:
ipd.Audio(x, rate=fs)

#### Percussive component based on KAM + NMF + Score-Informed Initialization 

In [ ]:
ipd.Audio(audios[0].T, rate=fs)

#### Harmonic component based on KAM + NMF + Score-Informed Initialization 

In [ ]:
ipd.Audio(audios[1].T, rate=fs)

#### Reference: 
[1] Christian Dittmar, Meinard Müller
<br>**Reverse Engineering the Amen Break — Score-Informed Separation and Restoration Applied to Drum Recordings**
<br>IEEE/ACM Transactions on Audio, Speech, and Language Processing, 24(9): 1531-1543, 2016.
<br>
[2] Christian Dittmar, Patricio López-Serrano, Meinard Müller
<br>**Unifying Local and Global Methods for Harmonic-Percussive Source Separation**
<br>In Proceedings of the IEEE International Conference on Acoustics,<br>Speech, and Signal Processing (ICASSP), 2018.

#### If you use the libnmfd (NMF toolbox) please refer to 
[3] Patricio López-Serrano, Christian Dittmar, Yiğitcan Özer, and Meinard Müller<br>
**NMF Toolbox: Music Processing Applications of Nonnegative Matrix Factorization**<br>
In Proceedings of the  International Conference on Digital Audio Effects (DAFx), 2019.